In [14]:
import matplotlib
# matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir

# 

import matplotlib.patches as mpatches


import glob2
import pandas as pd
from scipy import stats



# visualize results module
from visualize import visualize_svm as Visualize

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


labels = [
'left_paw',        # 0
'right_paw',       # 1
'nose',            # 2
'jaw',             # 3
'right_ear',       # 4
'tongue',          # 5
'lever',           # 6
'quiescence',      # 7
'code_04',         # 8
'code_04_lockout'  # 9
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [15]:

def compute_significance(data,
                         significance):

        #print ("self.data: ", data.shape)
        
        mean = data.mean(1)
        #
        sig = []
        for k in range(data.shape[0]):
            #res = stats.ks_2samp(self.data[k],
            #                     control)
            #res = stats.ttest_ind(first, second, axis=0, equal_var=True)

            #
            res = scipy.stats.ttest_1samp(data[k], 0.5)

            sig.append(res[1])


        sig_save = np.array(sig).copy()
        #print ("Self sig save: ", sig_save.shape)

        # multiple hypothesis test
        temp = np.array(sig)
        #print ("data into multi-hypothesis tes:", temp.shape)
        temp2 = multipletests(temp, alpha=significance, method='fdr_bh')
        sig = temp2[1]

        #
        sig=np.array(sig)[None]

        #
        thresh = significance
        idx = np.where(sig>thresh)
        sig[idx] = np.nan

        #
        idx = np.where(mean<0.5)
        sig[:,idx] = np.nan
        #print ("SIG: ", sig.shape)

        # find earliest 
        earliest_continuous = 0
        for k in range(sig.shape[1]-1,0,-1):
            if sig[0][k]<=significance:
                earliest_continuous = k
            else:
                break
                
        earliest_continuous = -(sig.shape[1]-earliest_continuous)/30.
                
        return sig, earliest_continuous
        

In [16]:
##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize.Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.lockout = False
vis.xvalidation = 10

vis.pca_var = 0.95
vis.pca_flag = True
vis.labels = labels

In [17]:
# #####################################
# ######## LOAD DEFAULT PARAMS ########
# #####################################
# vis.significance = 0.05
# vis.linewidth=10

# #
# vis.smooth_window = 10
# vis.lockout = False
# vis.xvalidation = 10
# vis.sliding_window = 30

# # 
# vis.window = 15
# vis.cbar_thick = 0.05
# vis.alpha = 1.0
    
# # 
# vis.min_trials = 10

# # 
# vis.animal_id= 'IA1'
# session = 'Feb1_'

# vis.pickle=True

# print ("   session: ", session)
# vis.session_id = session
# vis.cbar_offset = 0

# ############################

# codes = ['left_paw','right_paw','jaw']
# #code = 'left_paw'
# cmaps = ['Reds_r',"Blues_r","Greens_r"]
# clrs = ['red','blue','green']
# vis.show_title=False
# vis.show_legend=False
# vis.legend_location = 2
# vis.cbar=False
# vis.show_EDT = False
# vis.edt_offset = 0
# vis.linewidth = 1

# # alignment tools
# vis.show_all_times = True
# vis.shift_flag = True
# vis.shift = 0

# # 

In [18]:
#####################################################################################################
######## PLOT CORRELATION TRACES between lever and DLC traces for: left, right paw and lever ########
#####################################################################################################

# NOT PART OF FIGURES - USED FOR FIGURING OUT SHIFT BETWEEN DLC AND LEVER - DO NOT ERASE
# vis.animal_id = 'IJ2'

# sessions = np.loadtxt('/media/cat/4TBSSD/yuki/'+vis.animal_id+'/tif_files/sessions.txt',dtype='str')
# ctr=0
# labels = ['left_paw','right_paw','lever']
# fig=plt.figure()
# for session in sessions:
#     try:
#         data = np.load(session+'/correlate.npz',allow_pickle=True)
#         abspos = data['abspos_upsampled']
#         print (abspos.shape)
#         cor = data['cors'].squeeze().T
#         print ("cor: ", cor.shape)

#         ax=plt.subplot(7,10,ctr+1)
#         for k in range(3):
#             temp_trace = cor[:,k]
#             temp_trace[:2000] = 0
#             temp_trace[-2000:] = 0
#             temp_max = round(np.argmax(temp_trace)/1000. - 15.,2)
            
#             plt.plot(cor[:,k]/np.max(cor[:,k]),label=labels[k]+" "+str(temp_max))

#         plt.legend(fontsize=7)
        
#         plt.ylim(bottom=0)
#         plt.title(os.path.split(session)[1],fontsize=5)
#         ctr+=1
#         plt.xticks([])
#         plt.yticks(fontsize=6)
        
#     except:
#         pass
# plt.show()

In [19]:
#####################################################################
###### VISUALIZE ALL SESSIONS: [Ca] aligned to SELECTED SHIFT #######
#####################################################################

# NOT PART OF FIGS; - SHOWS ALL SESSIONS OF SVM DECODING FOR A SINGLE ANIMAL

# '''  This code plots several features for all sessions from each mouse;
#     - for single session plot and EDT plots see below

# '''


# vis.animal_id= 'AQ2'

# fnames_good = '/media/cat/4TBSSD/yuki/'+vis.animal_id + '/tif_files/sessions_DLC_alignment_good.txt'

# import csv
# sessions = []
# shift_ids = []
# with open(fnames_good, newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for row in spamreader:
#         sessions.append(str(row[0]))
#         shift_ids.append(row[1])

# ############################
# fig = plt.figure()
# ctr_plt = 0
# vis.pickle = True
# vis.xvalidation = 10
# vis.sliding_window = 30
# vis.smooth_window = 10
# vis.min_trials = 10
# vis.alpha = 1
# vis.significance = 0.05
# vis.ctr_plot=0

# # 
# for sess in sessions:
#     session = os.path.split(sess)[1].replace(vis.animal_id+'pm_','').replace('_30Hz','').replace(vis.animal_id+'am_','').replace('week','').replace(',','')
#     vis.session_id = session
#     vis.cbar_offset = 0

#     vis.ax = plt.subplot(7,10,ctr_plt+1)
#     codes = ['left_paw','right_paw','jaw']
#     cmaps = ['Reds_r',"Blues_r","Greens_r"]
#     clrs = ['red','blue','green']
    
#     # 
#     vis.show_title=False
#     vis.show_legend=False
#     vis.legend_location = 2
#     vis.cbar=False             # THIS SHOWS colorbar
#     vis.show_EDT = False
#     vis.edt_offset = 0
#     vis.linewidth = 1

#     # alignment tools
#     vis.xlim = [-5, 5]
#     vis.shift_flag = True
#     vis.shift_id_str = shift_ids[ctr_plt]
    
#     # 
#     vis.plotted= False # flag to indicate at least 1 plot was made to increase visual plot value
#     vis.n_trials_plotting = []
#     for ctr,code in enumerate(codes):
#         vis.ctr = ctr
#         vis.code = code

#         vis.cbar_offset=0.05*ctr
#         vis.cmap = cmaps[ctr]
#         vis.title = vis.code
#         vis.title_offset = .35
#         vis.plot_significant(clrs[ctr],vis.code)
#         #vis.cbar=True

#         #vis.ax.set_xticks([])

#     if vis.plotted:
#         plt.title(session+ " " + str(vis.n_trials_plotting), fontsize=10)
#         vis.ctr_plot+=1

#     ctr_plt+=1
    
#     print ('')
#     print ('')
    
#     #if ctr_plt>=2:
#     #    break
        

In [49]:
#######################################################################
###### VISUALIZE SINGLE SESSION: [Ca] aligned to SELECTED SHIFT #######
#######################################################################

# FIG 3J - SVM PLOT EXAMPLES FOR 1 MOUSE DECODING SEVERAL BODY PARTS

'''  This code plots several features for all sessions from each mouse;
    - for multi session plot see above
'''

vis.animal_id= 'IA3'         # Good mouse: IJ2 session Feb5_
vis.session_id = 'Feb15_'

# 
fnames_good = '/media/cat/4TBSSD/yuki/'+vis.animal_id + '/tif_files/sessions_DLC_alignment_good.txt'

# 
import csv
sessions = []
shift_ids = []
with open(fnames_good, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        sessions.append(str(row[0]))
        shift_ids.append(row[1])
print ("SESSIONS: ", sessions)

############################

labels = [
'left_paw',        # 0
'right_paw',       # 1
'nose',            # 2
'jaw',             # 3
'right_ear',       # 4
'tongue',          # 5
'lever',           # 6
'quiescence',      # 7
'code_04',         # 8
'code_04_lockout'  # 9
]

vis.ideal_window_flag = False
vis.dlc_offset_flag = False
vis.compute_roi = False
vis.imaging_rate = 30
vis.apply_lever_offset = False
vis.edt_offset_x = 0
vis.edt_offset_y = 0
vis.cbar_thick = .02
vis.shift_SVM = False

#
feature_ids = [0,1,5]
for id_ in feature_ids:

    vis.fig = plt.figure(figsize=(18,10))
    vis.ax = plt.subplot(1,1,1)

    ctr_plt = 0
    vis.pickle = True
    vis.xvalidation = 10
    vis.sliding_window = 30
    vis.smooth_window = 10
    vis.min_trials = 10
    vis.alpha = 1
    vis.significance = 0.05

    # 
    # session = 

    vis.cbar_offset = 0


    codes = labels[id_:id_+1]
    print ("CODES: ", codes)
    cmaps = ['Greys_r','Purples_r','Blues_r',"Greens_r","Reds_r",
             "Oranges_r","PuRd_r","PuBu_r",
            "YlGn_r","RdPu_r"]
    cmaps = cmaps[id_:id_+1]

    clrs = ['black','purple','blue','green','red','orange','darkred','darkgreen','lightgreen',
            'lightblue']
    clrs = clrs[id_:id_+1]

    # 
    vis.show_title=False
    vis.show_legend=False
    vis.legend_location = 2
    vis.cbar=False             # THIS SHOWS colorbar
    vis.show_EDT = True
    vis.edt_offset = 0
    vis.linewidth = 1

    # alignment tools
    vis.xlim = [-12, 0]
    vis.shift_flag = True
    vis.shift_id_str = shift_ids[ctr_plt]

    # 
    vis.plotted= False # flag to indicate at least 1 plot was made to increase visual plot value
    vis.n_trials_plotting = []
    edts = []
    for ctr,code in enumerate(codes):

        vis.ctr = ctr
        vis.code = code

        vis.cbar_offset=0.05*ctr
        vis.cmap = cmaps[ctr]
        vis.title = vis.code
        vis.title_offset = .35
        vis.plot_significant(clrs[ctr],vis.code)
        #vis.cbar=True

        #vis.ax.set_xticks([])

        edts.append(vis.earliest_continuous)
        print ('')
        print ('')
    #
    edts = np.array(edts)
    plt.suptitle(codes[0]+ " " + vis.session_id+ " " + str(vis.n_trials_plotting)+
              '\n EDTs: '+str(np.round(edts,2)), fontsize=15)

    # 
    if False:
        plt.savefig('/home/cat/'+codes[0]+'.svg')
        plt.close()
    else:
        plt.show()
        #if ctr_plt>=2:
    #    break
        

SESSIONS:  ['Feb1_,', 'Feb2_,', 'Feb3_,', 'Feb4_,', 'Feb9_,', 'Feb10_,', 'Feb11_,', 'Feb12_,', 'Feb15_,', 'Feb16_,', 'Feb17_,']
CODES:  ['left_paw']
PICKLE DATA:  10
self.code:  left_paw
Match code:  0
DATA;  (870, 10)
SELF SHIFT ID:  2
 using shift:  2
SHIFT Loaded:  4.82
... DLC shift applied for body movement:  4.82
 post trials data size  (870, 10)
      n-trials:  123
self n trials:  123
vmin, vmax;  0.0 0.05 ticks:  [0.         0.01666667 0.03333333 0.05      ]


CODES:  ['right_paw']
PICKLE DATA:  10
self.code:  right_paw
Match code:  1
DATA;  (870, 10)
SELF SHIFT ID:  2
 using shift:  2
SHIFT Loaded:  4.82
... DLC shift applied for body movement:  4.82
 post trials data size  (870, 10)
      n-trials:  95
self n trials:  95
vmin, vmax;  0.0 0.05 ticks:  [0.         0.01666667 0.03333333 0.05      ]


CODES:  ['tongue']
PICKLE DATA:  10
self.code:  tongue
Match code:  5
DATA;  (870, 10)
SELF SHIFT ID:  2
 using shift:  2
SHIFT Loaded:  4.82
... DLC shift applied for body movemen

In [41]:
#########################################################################
###### SCATTER PLOTS EDT FOR ALL MICE ALL SESSIONS ALL BODY PARTS #######
#########################################################################

# FIG 3K - Scatter plots of EDT
codes = [
'left_paw',          # 0
'right_paw',         # 1
'nose',              # 2
'jaw',               # 3
'right_ear',         # 4
'tongue',            # 5
# 'lever',           # 6
# 'quiescence',      # 7
# 'code_04',         # 8
# 'code_04_lockout'  # 9
]


animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

# 
for animal_id in animal_ids: 
    
    # 
    vis = Visualize.Visualize()
    vis.shift_flat = True
    vis.pickle = True
    vis.compute_roi = False  # flag for computing data only based on ROIs.

    # lever-related data
    vis.main_dir = data_dir
    vis.random_flag = False  # shuffle data to show baseline

    # 
    vis.window = 15
    vis.lockout_window = 10
    vis.lockout = False
    vis.xvalidation = 10
    vis.sliding_window = 30
    vis.min_trials = 30
    vis.smooth_window = 30
    
    # 
    vis.significance = 0.05
    vis.pca_var = 0.95
    vis.pca_flag = True
    vis.labels = labels
    vis.dlc_offset_flag = True
    vis.shift_flag = True
    vis.apply_lever_offset = True

    vis.animal_id= animal_id

    # 
    fnames_good = '/media/cat/4TBSSD/yuki/'+vis.animal_id + '/tif_files/sessions_DLC_alignment_good.txt'

    import csv
    sessions = []
    shift_ids = []
    with open(fnames_good, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            sessions.append(str(row[0]).replace(',',''))
            shift_ids.append(row[1])

    print ("SESSIONS: ", sessions)
    print ("SHIFT IDS: ", shift_ids)

    ############################

    #
    edt = []
    n_trials = []

    # loop over all sessions
    for ctr,session_id in enumerate(sessions):
        edt.append([])
        n_trials.append([])
        # get the shifted data by matching session ID with all sessions
        vis.session_id = session_id
        for k in range(len(sessions)):
            if vis.session_id in sessions[k]:
                ctr_plt = k
                break

        # loop over all limb movements
        vis.xlim = [-15, 0]
        for code in codes:

            vis.code = code
            vis.shift_id_str = shift_ids[ctr_plt]

            # GET FILENAME IF EXISTS
            vis.get_fname()
            if vis.fname is None:
                print ("no file, exiting")
                edt[ctr].append(np.nan)
                n_trials[ctr].append(0)
                continue

            # PROCESS SESSION
            vis.process_session()
            if vis.n_trials==0 or vis.data.shape[0]==0:
                print ("insufficient trials ", code)
                edt[ctr].append(np.nan)
                n_trials[ctr].append(0)
                continue

            # GET SIGNIFICANCE AND EDT
            vis.compute_significance()
            edt[ctr].append(vis.edt)
            n_trials[ctr].append(vis.n_trials)
            print ("*******************************")
            print ("    EDT: ", code, " ", vis.edt)
            print ("*******************************")
            print ('')
            print ('')

    n_trials = np.array(n_trials)
    edt = np.array(edt)
    print (edt.shape)
    
    # data_dir = '/media/cat/4TBSSD/yuki/'
    np.savez(data_dir + animal_id+"_edt_body_parts.npz",
            edt = edt,
            n_trials = n_trials)


SESSIONS:  ['Feb1_', 'Feb2_', 'Feb3_', 'Feb5_', 'Feb9_', 'Feb10_', 'Feb11_', 'Feb12_', 'Feb15_', 'Feb16_', 'Feb17_', 'Apr21_', 'Apr25_', 'Apr26_', 'Apr27_', 'Apr28_', 'Apr29_', 'May6_', 'May9_', 'May10_', 'May12_', 'May17_', 'May18_', 'May20_']
SHIFT IDS:  ['2', '1', '2', '2', '2', '2', '2', '0', '2', '2', '1', '0', '0', '1', '1', '1', '1', '2', '0', '2', '2', '2', '02', '0']
PICKLE DATA:  10
self.code:  left_paw
Match code:  0
DATA;  (870, 10)


AttributeError: 'Visualize' object has no attribute 'ideal_window_flag'

In [21]:
##############################
### GENERATE SCATTER PLOT ####
##############################

# FIG 3K - Scatter plots of EDT
clrs = ['black','purple','blue','green','red','orange']

# 
animal_id = 'IA1'
data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'_edt_body_parts.npz')
edt = data['edt']
n_trials = data['n_trials']

#
fig = plt.figure(figsize=(10,20))
for k in range(edt.shape[1]):
    ax=plt.subplot(2,3,k+1)
    
    idx = np.where(np.logical_and(edt[:,k]!=0,np.isnan(edt[:,k])==False))[0]
    t = np.array(n_trials[idx,k])
    temp = np.array(edt[idx,k])
    
    # 
    plt.scatter(temp,t,c=clrs[k],
               s=500,edgecolor='black',alpha=1-np.arange(temp.shape[0])/temp.shape[0])
    
    # 
    coef = np.polyfit(temp,t,1)
    poly1d_fn = np.poly1d(coef) 
    plt.plot(temp, poly1d_fn(temp), c='black',
            linewidth=5,alpha=.5)    

    # 
    plt.xlim(-15,0)
    plt.ylim(0,200)
    plt.title(codes[k])
    plt.xticks([])
    plt.yticks([])
    
if False:
    plt.savefig('/home/cat/scatters_bpodies.png',dpi=300)
    plt.close()
else:
    plt.show()
    


In [54]:
###########################################################
###### BOX PLOTS FOR ALL BODY MOVEMENTS ALL ANIMALS #######
###########################################################

# 
def plot_edt_distributions_box_plots():

    codes = [
             'left_paw',          # 0
             'right_paw',         # 1
             'nose',              # 2
             'jaw',               # 3
             'right_ear',         # 4
             'tongue'
            ]

    #codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
    clrs_local = ['grey','magenta','blue','green','red','orange']
    codes = [
    'left_paw',          # 0
    'right_paw',         # 1
    'nose',              # 2
    'jaw',               # 3
    'right_ear',         # 4
    'tongue']

    
    
    vis = Visualize.Visualize()
    
    fig = plt.figure(figsize=(20,12))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(2,3,ctr+1)
        fnames = []
        
        #for code in codes:
        fname = '/media/cat/4TBSSD/yuki/'+animal_id+'_edt_body_parts.npz'

        # 
        #shifts=[+15,+15,+15,+15,+15,+15,+15]
        shifts=[0,0,0,0,0,0,0]
        linestyles = ['-','-','-','-','-','-','-','-']
        #titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []

        data = np.load(fname, allow_pickle=True)
        data = data['edt']

        for ctr_type in range(data.shape[1]):
            # 

#             # 
#             clr = vis.colors[ctr_type]
#             plotting = False
#             res = vis.compute_first_decoding_time_curves(data[ctr_type],
#                                                          shifts[ctr_type])
            

            temp = data[:,ctr_type]
            print ("temp: ", temp)
            # remove low values; don't use fo rnow
            if False:
                idx = np.where(temp<-0.5)[0]
                temp=temp[idx]

            #    
            edts.append(temp)
            #yvals.append(res[1])

        edts_saved = np.array(edts).copy()
        print ("****************************************")
        print (" # of EDTS: ", len(edts))
        print ("edts: ", edts)
        my_dict = dict(left_paw = edts[0], 
                       right_paw = edts[1],
                       nose = edts[2],
                       ear = edts[3],
                       jaw = edts[4],                      
                       tongue = edts[5],                      
                      )
        
        data = pd.DataFrame.from_dict(my_dict, orient='index')
        data = data.transpose()

        # 
        flierprops = dict(marker='o', 
                          #markerfacecolor='g', 
                          #markersize=10000,
                          linestyle='none', 
                          markeredgecolor='r')
        
        #
        data.boxplot(showfliers=False,
                    flierprops=flierprops)
        
        
        for i,d in enumerate(data):
            y = data[d]
            x = np.random.normal(i+1, 0.04, len(y))
            plt.plot(x, y, 
                     mfc =clrs_local[i], 
                     mec='k', 
                     ms=7, 
                     marker="o", 
                     linestyle="None",
                     
                    )
            
#             plt.scatter(x, y, 
#                       c=clrs_local[i],
#                        edgecolor='black',
#                        s=200,
#                        alpha=.7)

            
        if animal_id=='IA1':
            patches = []
            for c,code in enumerate(codes):
                patches.append(mpatches.Patch(color=clrs_local[c], label=code))
        
        
            plt.legend(handles=patches,fontsize=legend_size,
                      ncol=2)

       # temp = np.hstack(yvals)
       # ymax = np.max(temp)

        #
        plt.xlim(0.5, 6.5)
        plt.ylim(-15,0)    
        plt.xticks([])
        plt.yticks([])

        #
        plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
        plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)

        # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
        if False:
            patches = []
            for p in [0,2,4]:
                res = stats.ks_2samp(edts_saved[p], edts_saved[3])
                label_ = ''
                for k in range(len(pvals)):
                    if res[1]<pvals[k]:
                        label_ = label_ + "*"
                    else:
                        break

                patches.append(mpatches.Patch(color='blue', label=label_))


            #print ('')
            ax2 = ax.twinx()  
            ax2.legend(handles=patches,
                       fontsize=legend_size)
            plt.yticks([])


    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first_scatter_body_movements.svg')
        plt.close()
        
###
plot_edt_distributions_box_plots()

# # 
# Visualize.plot_vertical_histograms(animal_ids,
#                                    code_ids,
#                                    clrs,
#                                    window,
#                                     labels,
#                                     -10.0)

#plt.savefig('/home/cat/'+str(code_ids)+'.png',dpi=300)
#plt.close()
#
#plt.show()
    

temp:  [-4.53333333 -3.03333333  0.         -2.56666667 -3.23333333 -3.1
 -6.53333333 -3.26666667 -4.         -3.6        -4.1        -4.26666667
 -3.83333333  0.         -1.33333333 -0.9        -1.4         0.
 -3.1        -3.06666667 -1.9        -2.93333333 -2.56666667 -3.06666667]
temp:  [-4.9         0.         -2.06666667 -3.33333333 -2.83333333 -2.56666667
 -4.         -2.8        -2.86666667 -6.2        -2.9        -5.23333333
 -2.93333333  0.          0.         -1.36666667  0.          0.
 -2.86666667 -3.43333333  0.         -2.46666667 -2.83333333 -1.5       ]
temp:  [-3.43333333  0.         -3.03333333         nan         nan -4.4
 -3.63333333 -3.6        -3.23333333 -4.63333333 -5.03333333 -4.76666667
 -3.3         0.         -0.5         0.         -2.          0.
  0.         -2.36666667 -0.13333333 -2.66666667 -3.1        -2.23333333]
temp:  [-2.73333333  0.         -1.8         0.         -1.5        -2.6
 -3.26666667 -3.93333333  0.         -3.6        -0.83333333 -4.5

In [28]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2_edt_body_parts.npz', allow_pickle=True)
edt = data['edt']
print (edt.shape)



(53, 6)


In [2]:
#####################################
####### PLOT PREDICTOIN TRENDS ######
#####################################

        
names = ['IA1','IA2','IA3','IJ1','IJ2', 'AQ2']

fig, ax = plt.subplots()
import scipy
from statsmodels.stats.multitest import multipletests

smooth = True
smooth_window = 6
significance = 0.05
auc = []
early = []
from tqdm import tqdm, trange
for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    auc.append([])
    early.append([])
    for ctr,fname in tqdm(enumerate(fnames)):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)
            data = np.array(data)
            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        #print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())
        
        sig, earliest = compute_significance(data.T,
                                             significance)
        early[k].append(earliest)
        
    plt.ylim(0.4,1.0)
    plt.xlim(-15,0)
    plt.plot([-30,0],[0.5,0.5],'--',c='black')

plt.show()

46it [00:14,  3.26it/s]
33it [00:10,  3.26it/s]
41it [00:12,  3.26it/s]
40it [00:12,  3.26it/s]
39it [00:11,  3.25it/s]
78it [00:24,  3.25it/s]


In [6]:
#########################################
########### AUC AND EARLY PLOTS #########
#########################################

# FIg 3J 3I

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig2=plt.figure()
for k in range(len(early)):
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])
    
    #
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             label=ids[k]+ " ***** ",

             c=colors[k])

    # 
    corr = scipy.stats.pearsonr(t,temp2)
    print ("corr: ", corr)
    
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                linewidth=4,
                c=colors[k],
                alpha=.3)
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)


#
plt.xlim(0,80)
plt.ylim(-12,0)
plt.legend(fontsize=20)
plt.show()


corr:  (1.0, 0.0)
corr:  (-0.9999999999999999, 3.3310483861078736e-244)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-0.9999999999999999, 0.0)


In [196]:
#############################################################
### COMPUTE DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ##
#############################################################
from tqdm import trange

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 30
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False
vis.imaging_rate = 30.
vis.pca_var = 0.95
vis.pca_flag = True
vis.xvalidation = 10
vis.sliding_window = 30

# 
vis.smooth_window = 30
vis.min_trials = 20

#
vis.significance=1E-2

# if already computed
if False:
    #lockouts = [False, True]
    lockouts = [False]
    vis.compute_first_decoding_time(lockouts)

# if already computed
if True:
    #lockouts = [False, True]
    lockouts = [False]
    vis.compute_first_decoding_time_concatenated(lockouts)



  0%|          | 0/6 [00:00<?, ?it/s]

 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  

Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_May13_Week5_30Hz/IA1am_May13_Week5_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_May16_Week6_30Hz/IA1am_May16_Week6_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data in

 17%|█▋        | 1/6 [00:17<01:28, 17.70s/it]

Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
 ... data missing /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb1_30Hz/IA2pm_Feb1_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb2_30Hz/IA2pm_Feb2_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
 ... data missing /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb9_30Hz/IA2pm_Feb9_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb10_30Hz/IA2pm_Fe

...

Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/AQ2am_Dec9_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec10_30Hz/AQ2am_Dec10_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec10_30Hz/AQ2pm_Dec10_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec11_30Hz/AQ2am_Dec11_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec14_30Hz/AQ2pm_Dec14_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec14_30Hz/AQ2am_Dec14_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
L

Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
LOADED: DATA (870, 10

100%|██████████| 6/6 [01:33<00:00, 15.56s/it]

Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)



/media/cat/4TBSSD/anaconda3/envs/wide/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [209]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# return all session ids that can be predicted longer than this:
return_ids_threshold = -10

vis.window = 30
vis.lockout_window = 10

# 
vis.min_trials = 20  #min trials used for SVM

vis.lockout=False
# 
clrs = ['blue','green']
vis.plot_first_decoding_time(return_ids_threshold, clrs)

# 
if True:
    plt.ylim(-15,0)
    plt.show()
else:
    plt.savefig('/home/cat/first.png',dpi=600)
    plt.close()

all:  ['AQ2am_Feb12_30Hz' 'AQ2am_Feb15_30Hz']
all:  [60 61]
[[-2.8000000000000007, -7.266666666666666, -3.7333333333333343, -2.366666666666667, -3.033333333333335, -3.1999999999999993, -4.033333333333335, -3.466666666666665, -2.2666666666666657, -1.2333333333333343, -3.666666666666668, -2.533333333333335, -2.866666666666667, -2.966666666666665, -2.0666666666666664, -1.0, -2.0666666666666664, -3.2333333333333343, -2.333333333333332, -2.333333333333332, -1.9333333333333336, -1.0, -2.2666666666666657, -1.5666666666666664, -2.4333333333333336, -2.0666666666666664, -1.466666666666665, -1.1999999999999993, -2.0666666666666664, -2.0, -1.6999999999999993], [-1.0, -4.300000000000001, -3.7666666666666657, -1.0, -2.966666666666665, -1.0, -2.166666666666668, -1.5, -1.0, -1.0, -2.1000000000000014, -1.0, -1.533333333333335, -1.2666666666666657, -1.0, -1.0, -4.533333333333335, -4.533333333333335, -3.7666666666666657, -4.266666666666666, -3.7333333333333343, -5.300000000000001, -4.133333333333333, -4.

In [205]:
############################################################
#### PLOT CORRELATION OF DECODING TIME AND NO. SESSIONS ####
############################################################
from tqdm import trange

vis.smooth_window = 30
vis.min_trials = 20

vis.window = 30
vis.xlim=-15

#
vis.significance=0.05 #1E-2
vis.lockout=False

# 
clr='red'
fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_lockout_30sec_concatenated_backup.npz'
vis.plot_first_decoding_time_vs_n_trials(clr, fname)

# #
if False:
    plt.show()
else:
    plt.savefig('/home/cat/first.png',dpi=600)
    plt.close()


AQ2 AQ2am_Apr29_Week8_30Hz [ -4.2         -6.63333333  -4.36666667  -4.63333333  -4.86666667
 -11.43333333 -10.13333333  -6.66666667  -4.43333333  -5.06666667
  -4.2         -4.16666667  -4.33333333  -4.4         -6.33333333
  -4.7         -3.86666667  -4.3         -3.56666667  -3.9
  -3.36666667  -3.96666667  -3.56666667  -3.73333333  -3.86666667
  -3.63333333  -4.26666667  -3.3         -2.86666667  -3.96666667
  -3.66666667  -4.13333333  -3.36666667  -3.26666667  -2.83333333
  -3.63333333  -3.03333333  -3.          -2.7         -3.
  -2.9         -2.66666667  -3.6         -2.43333333  -3.
  -2.46666667  -2.53333333  -2.5         -2.5         -2.16666667
  -2.          -2.          -2.        ]
all preds:  (53, 1)
AQ2 AQ2am_Apr29_Week8_30Hz [-4.53333333 -4.33333333 -3.13333333 -2.63333333 -2.66666667 -2.
 -2.8        -2.4        -3.13333333 -3.66666667 -2.         -2.
 -2.         -4.96666667 -4.8        -4.83333333 -4.76666667 -5.2
 -8.63333333 -7.36666667 -7.4        -7.2        -7.

In [195]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar31_30Hz/IA1pm_Mar31_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz',
              allow_pickle=True)
acc= data['accuracy']
labels = data['labels']
print (acc.shape)
print (labels.shape)

(870, 10)
(870, 436)
